In [1]:
%pip install osmium folium geodatasets


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import folium
import osmium as osm
import pandas as pd
import urllib.request
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon

In [3]:
#you can create shape here https://clydedacruz.github.io/openstreetmap-wkt-playground/
#copy polygon into var wkt_poly

#seattle downtown
# wkt_poly = '-122.3567 47.5933,-122.3139 47.5933,-122.3139 47.6517,-122.3567 47.6517'

#new york manhattan downtown
wkt_poly = '-74.00239853421225 40.76848093731104,-73.95889190956949 40.752874609831736,-73.97423808462916 40.70989961182363,-73.99736844934523 40.704987174650086,-74.01250212220474 40.69698290095204,-74.0223849727772 40.69959263102922,-74.02202142868191 40.71276213012328,-74.01762143243103 40.73881298686783,-74.01512450538576 40.7547003536466,-74.00239853421225 40.76848093731104'

poly_points = [(float(i.split(' ')[1]), float(i.split(' ')[0])) for i in wkt_poly.split(',')]

current_dir = os.getcwd()
filename = f'{current_dir}/map-data.osm'

In [4]:
map_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [[[i[1], i[0]] for i in poly_points]],
        "type": "Polygon"
      }
    }
  ]
}
m = folium.Map()
geo_j = folium.GeoJson(data=map_data, style_function=lambda x: {"fillColor": "orange"})
folium.Popup('location').add_to(geo_j)
geo_j.add_to(m)
m.fit_bounds([[min([i[0] for i in poly_points]), min([i[1] for i in poly_points])], [max([i[0] for i in poly_points]), max([i[1] for i in poly_points])]]) 
m

<h1>Download map data</h1>

In [5]:
print(f"wkt polygon:POLYGON(({','.join([f'{i[1]} {i[0]}' for i in poly_points])}))")
poly = ' '.join([f'{i[0]} {i[1]}' for i in poly_points])
url = f'''http://overpass-api.de/api/interpreter?data=(node(poly:"{poly}");way(poly:"{poly}"););(._;>;);out;'''
url = url.replace(" ", "%20")
%time urllib.request.urlretrieve(url, filename)

wkt polygon:POLYGON((-74.00239853421225 40.76848093731104,-73.95889190956949 40.752874609831736,-73.97423808462916 40.70989961182363,-73.99736844934523 40.704987174650086,-74.01250212220474 40.69698290095204,-74.0223849727772 40.69959263102922,-74.02202142868191 40.71276213012328,-74.01762143243103 40.73881298686783,-74.01512450538576 40.7547003536466,-74.00239853421225 40.76848093731104))
CPU times: user 146 ms, sys: 222 ms, total: 369 ms
Wall time: 9.16 s


('/Users/chaoqinglu/src/office_locations/map-data.osm',
 <http.client.HTTPMessage at 0x126765ed0>)

<h1>Process map data</h1>

In [6]:
valid_building_tags = set(['office', 'commercial', 'university', 'school', 'college', 'bank'])
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.node_coordinates = {}
        self.missing_nodes = 0

    def tag_office(self, elem, elem_type):
        name = ''
        tag_key_office = ''
        tag_val_office = ''
        is_building = False
        building_tag_val = ''
        
        for tag in elem.tags:
            if tag.k.lower() == 'website':
                continue
            if 'office' in tag.k.lower() and tag_key_office != 'office':
                tag_key_office = tag.k.lower()
                tag_val_office = tag.v.lower()
            if 'office' in tag.v.lower() and tag_val_office != 'office':
                tag_key_office = tag.k.lower()
                tag_val_office = tag.v.lower()
            if ('building'==tag.k):
                is_building = True
                building_tag_val = tag.v.lower()
            if tag.k == 'name':
                name = tag.v
        if tag_key_office != '' or tag_val_office != '' or \
            (is_building and (
                building_tag_val in valid_building_tags or \
                building_tag_val == 'yes' and name != ''
            )):
            points = []
            if elem_type == 'way':
                for n in elem.nodes:
                    if n.ref in self.node_coordinates:
                        points.append(self.node_coordinates[n.ref])
                    else:
                        self.missing_nodes += 1
                        #print(f'missing loc for {n.ref} {elem.id}')
            else:
                points.append((elem.location.lon, elem.location.lat))
            self.osm_data.append([
                elem_type, 
                int(elem.id), 
                name.lower(),
                tag_key_office,
                tag_val_office,
                is_building, 
                building_tag_val,
                ','.join([f'{tag.k.lower()}:{tag.v.lower()}' for tag in elem.tags]),
                points,
            ])

    def node(self, n):
        self.node_coordinates[n.id] = (n.location.lon, n.location.lat)
        self.tag_office(n, "node")
    def way(self, w):
        self.tag_office(w, "way")


osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file(filename)
print(f'missing {osmhandler.missing_nodes} nodes in ways')
# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'name', 'tag_key', 'tag_val', 'is_building', 'building_tag', 'tags', 'points']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
df_osm.sort_values(by=['name'])

missing 0 nodes in ways


,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points
1010,way,249657366,,building,office,True,office,"building:office,height:79.7,nycdoitt:bin:1017860","[(-73.9880672, 40.7365718), (-73.9882058, 40.7..."
836,way,247861546,,building,office,True,office,"addr:housenumber:39,addr:postcode:10003,addr:s...","[(-73.9917222, 40.7335667), (-73.9918775, 40.7..."
835,way,247861544,,building,office,True,office,"addr:housenumber:37,addr:postcode:10003,addr:s...","[(-73.9920418, 40.7336898), (-73.9921597, 40.7..."
1445,way,265424356,,,,True,commercial,"addr:city:new york,addr:housenumber:9,addr:pos...","[(-73.9821917, 40.7500222), (-73.9823621, 40.7..."
830,way,247858039,,,,True,university,"addr:housenumber:64,addr:postcode:10003,addr:s...","[(-73.9960076, 40.7317308), (-73.9960594, 40.7..."
...,...,...,...,...,...,...,...,...,...
283,node,3510429862,yext,office,company,False,,"addr:city:new york,addr:housenumber:61,addr:po...","[(-74.0055193, 40.741793)]"
1866,way,278256429,young israel,,,True,yes,"addr:city:new york,addr:housenumber:519,addr:p...","[(-73.9818959, 40.7143431), (-73.981952, 40.71..."
699,way,158413988,zeckendorf towers,building,office,True,office,"building:office,building:colour:#cca77d,buildi...","[(-73.9884412, 40.7346337), (-73.9884829, 40.7..."
386,node,5618806821,zocdoc,office,company,False,,"addr:city:new york,addr:housenumber:568,addr:p...","[(-73.9973808, 40.7243026)]"


<h1>QAs</h1>

In [7]:
df_osm[['type', 'tag_key', 'tag_val', 'is_building']].value_counts()[:100]

type  tag_key         tag_val                                                                                              is_building
way                                                                                                                        True           1169
node  office          diplomatic                                                                                           False           159
way   building        office                                                                                               True            145
node  office          company                                                                                              False            91
                                                                                                                           True             61
                                                                                                                                          ... 
      e

In [8]:
df_osm['name'].value_counts()[:100]

                             249
fedex office                  14
united states post office     13
the ups store                  8
wework                         8
                            ... 
rick's cabaret                 1
lucy's cantina royale          1
sir stage37                    1
doxie hotel                    1
world's tower building         1
Name: name, Length: 100, dtype: int64

In [9]:
df_osm[df_osm.apply(lambda x:'google' in x['name'] or 'google' in x['tags'], axis=1)]

,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points
378,node,5525153732,"mypostcard.com, inc.",office,company,False,,"addr:city:new york,addr:housenumber:433,addr:p...","[(-74.0015682, 40.7202256)]"
874,way,247866662,moma design store,,,True,yes,"addr:housenumber:81,addr:postcode:10012,addr:s...","[(-73.9980098, 40.7227517), (-73.9981638, 40.7..."
1187,way,264624898,111 eighth avenue,building,office,True,office,"addr:city:new york,addr:state:ny,alt_name:port...","[(-74.002127, 40.7405852), (-74.002242, 40.740..."


In [10]:
df_osm[df_osm['id']==512681243]

,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points


<h1>Lets do some dedupping if we think its necessary, if a WAY has a node that is already included in the data set, remove the WAY</h1>
Note: this may under estimate the num of offices

In [11]:
dup_way_and_nodes = []
for _, way in df_osm[df_osm['type'] == 'way'].iterrows():
#     if way.id != 512681243:
#         continue
    if len(way.points) == 0:
        continue
    for _, node in df_osm[df_osm['type'] == 'node'].iterrows():
#         if node.id != 10653099469:
#             continue
        polygon = Polygon(way.points)
        point = Point(node.points[0][0], node.points[0][1])
        if polygon.contains(point):
            dup_way_and_nodes.append([way.id, node.id])
df_way_and_nodes = pd.DataFrame(dup_way_and_nodes, columns=['wid', 'nid'], dtype='int')

In [12]:
df_way_and_nodes[df_way_and_nodes['wid']==264624898]

,wid,nid
115,264624898,1825739869


In [13]:
dup_ways = set(df_way_and_nodes['wid'].unique())

In [14]:
df_offices = df_osm[
    df_osm[['type','id']].apply(
        lambda x:True if x['type']=='node' else True if x['id'] not in dup_ways else False, 
        axis=1
    )
]
df_offices

,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points
0,node,357610837,nyc4ukraine,office,charity,False,,"name:nyc4ukraine,office:charity,operator:corne...","[(-73.9900888, 40.7289704)]"
1,node,357620365,the new york foundling,office,ngo,False,,"addr:city:new york,addr:housenumber:590,addr:p...","[(-73.9954239, 40.7389359)]"
2,node,368042962,metro communications center,,,True,yes,"addr:state:ny,building:yes,ele:10,gnis:county_...","[(-73.9980648, 40.7144331)]"
3,node,368042971,kearney and van tassel auction stables,,,True,yes,"addr:state:ny,building:yes,ele:12,gnis:county_...","[(-73.9891667, 40.7327778)]"
4,node,368042980,storefront for art and architecture,,,True,yes,"addr:housenumber:97,addr:postcode:10012,addr:s...","[(-73.9977778, 40.7213889)]"
...,...,...,...,...,...,...,...,...,...
2017,way,1112436284,flat iron plate,,,True,yes,"addr:city:new york,addr:postcode:10010,addr:st...","[(-73.9893279, 40.7411418), (-73.9893425, 40.7..."
2018,way,1127609567,titanic memorial lighthouse,,,True,yes,"building:yes,height:18,historic:memorial,mater...","[(-74.003882, 40.7073984), (-74.0038816, 40.70..."
2019,way,1128358845,robert and anne dickey house,,,True,yes,"addr:city:new york,addr:housenumber:67,addr:po...","[(-74.0137756, 40.7072898), (-74.0138251, 40.7..."
2020,way,1139026049,african grove theatre,,,True,yes,"addr:city:new york,addr:housenumber:181,addr:p...","[(-73.9977191, 40.7265195), (-73.9975731, 40.7..."


In [15]:
df_offices[df_offices.apply(lambda x:'chase' in x['name'] or 'chase' in x['tags'], axis=1)]

,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points
477,node,6910663884,chase global realty manhattan,office,estate_agent,False,,"name:chase global realty manhattan,office:esta...","[(-73.9954407, 40.716271)]"
681,way,146278002,jp morgan chase tower,,,True,commercial,"addr:housenumber:270,addr:street:park avenue,a...","[(-73.9757079, 40.7561267), (-73.9757704, 40.7..."
1897,way,279530782,28 liberty,building,office,True,office,"addr:city:new york,addr:housenumber:28,addr:po...","[(-74.0092146, 40.7082059), (-74.0092229, 40.7..."
